# 使用pytorch 构建网络模型

In [1]:
import numpy as np
import torch as t
#生成噪声和数据
def synthetic_data(w: t.Tensor, b: float, num_examples: int):
    x = t.normal(0, 1, (num_examples, len(w)))
    y = t.matmul(x, w)+b
    y += t.normal(0, 0.01, y.shape)
    return x, y.reshape((-1, 1))


true_w = t.tensor([2., -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print("features:{},\n labels:{}".format(features[0], labels[0]))



features:tensor([0.5205, 0.1084]),
 labels:tensor([4.8784])


# 使用t.utils.data.DataLoader实现迭代器。

In [2]:
import torch.utils.data as data
def load_array(data_arrays,batch_size,is_train=True):
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size=10
data_iter=load_array((features,labels),batch_size)
next(iter(data_iter))


[tensor([[-0.7703,  0.6876],
         [ 1.6071,  0.7902],
         [-0.2983, -1.3070],
         [ 0.6706,  0.3865],
         [-0.0572, -0.5454],
         [ 1.4626,  1.4208],
         [-1.1732,  0.8220],
         [-0.0059, -2.6996],
         [-1.3600, -1.6149],
         [-0.2509, -0.9998]]),
 tensor([[ 0.3227],
         [ 4.7143],
         [ 8.0351],
         [ 4.2322],
         [ 5.9518],
         [ 2.3251],
         [-0.9419],
         [13.3800],
         [ 6.9714],
         [ 7.0932]])]

In [5]:
import torch.nn as nn
#就相当于前面的手写的例子
net  =  nn.Sequential(nn.Linear(2,1))
# 进行一个权重的初始化
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)
loss= nn.MSELoss()
trainer=t.optim.SGD(net.parameters(),lr=0.003)
num_epochs=15
for epoch in range(num_epochs):
    for x,y in data_iter:
        l=loss(net(x),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l=loss(net(features),labels)
    print(f'epoch:{epoch},loss:{l:f}')

epoch:0,loss:9.874764
epoch:1,loss:2.911777
epoch:2,loss:0.859588
epoch:3,loss:0.253977
epoch:4,loss:0.075174
epoch:5,loss:0.022310
epoch:6,loss:0.006678
epoch:7,loss:0.002051
epoch:8,loss:0.000681
epoch:9,loss:0.000275
epoch:10,loss:0.000155
epoch:11,loss:0.000119
epoch:12,loss:0.000108
epoch:13,loss:0.000105
epoch:14,loss:0.000104


# 查看网络的参数和偏置（bias）

In [12]:
for name,param in net.named_parameters():
    print(name,param)
    print(param.grad_fn)

0.weight Parameter containing:
tensor([[ 1.9999, -3.3994]], requires_grad=True)
None
0.bias Parameter containing:
tensor([4.1997], requires_grad=True)
None
